# Reference frames

rigid_body_motion provides a flexible high-performance framework for working offline with motion data. The core of this framework is a mechanism for constructing trees of both static and dynamic reference frames that supports automatic lookup and application of transformations across the tree.

In [ ]:
import numpy as np
import rigid_body_motion as rbm

<div class="alert alert-info">
Note
    
The following examples require the `matplotlib` library.
</div>

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (6, 6)

## Static frames

We will begin by defining a world reference frame using the [ReferenceFrame](_generated/rigid_body_motion.ReferenceFrame.rst) class:

In [ ]:
rf_world = rbm.ReferenceFrame("world")

Now we can add a second reference frame as a child of the world frame. This frame is translated by 5 meters in the x direction and rotated 90° around the yaw axis. Note that rotations are represented as [unit quaternions](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation) by default.

In [ ]:
rf_observer = rbm.ReferenceFrame(
    "observer",
    parent=rf_world,
    translation=(5, 0, 0),
    rotation=(np.sqrt(2) / 2, 0, 0, np.sqrt(2) / 2),
)

We can show the reference frame tree with the [render_tree](_generated/rigid_body_motion.render_tree.rst) function:

In [ ]:
rbm.render_tree(rf_world)

It is also possible to show a 3d plot of static reference frames with [plot.reference_frame()](_generated/rigid_body_motion.plot.reference_frame.rst):

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

rbm.plot.reference_frame(rf_world, ax=ax)
rbm.plot.reference_frame(rf_observer, rf_world, ax=ax)

To facilitate referring to previously defined frames, the library has a registry where frames can be stored by name with [ReferenceFrame.register()](api/_generated/rigid_body_motion.ReferenceFrame.register.rst):

In [ ]:
rf_world.register()
rf_observer.register()
rbm.registry

## Transforming positions and orientations

Now that we've set up a basic tree, we can use it to transform motion between reference frames. We use the [lookup_transform()](_generated/rigid_body_motion.lookup_transform.rst) method to obtain the transformation from the world to the observer frame:

In [ ]:
t, r, ts = rbm.lookup_transform(outof="world", into="observer")

This transformation consists of a translation $t$:

In [ ]:
t

and a rotation $r$:

In [ ]:
r

The method also returns the timestamps of the transformation; in the case of a static transform these are `None`.

rigid_body_motion uses the convention that a transformation is a rotation followed by a translation. Here, when applying the transformation to a point $p$ expressed with respect to (wrt) the world frame $W$ it yields the point wrt the observer frame $O$:

$$p_O = \operatorname{rot}\left(r, p_W\right) + t$$

The $\operatorname{rot}()$ function denotes the [rotation of a vector by a quaternion](https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation#Using_quaternions_as_rotations).

Let's assume we have a rigid body located at 2 meters in the x direction from the origin of the world reference frame:

In [ ]:
p_body_world = np.array((2, 0, 0))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

rbm.plot.reference_frame(rf_world, ax=ax)
rbm.plot.reference_frame(rf_observer, rf_world, ax=ax)
ax.plot(p_body_world[0], p_body_world[1], p_body_world[2], "yo")

We can use the above formula to transform the position of the body into the observer frame. The [rotate_vectors()](_generated/rigid_body_motion.rotate_vectors.rst) method implements the rotation of a vector by a quaternion:

In [ ]:
p_body_observer = rbm.rotate_vectors(r, p_body_world) + t
p_body_observer

As expected, the resulting position of the body is 3 meters from the observer in the y direction. For convenience, the [transform_points()](_generated/rigid_body_motion.transform_points.rst) method performs all of the above steps:

1. Lookup of the frames by name in the registry (if applicable)
2. Computing the transformation from the source to the target frame
3. Applying the transformation to the point(s)

In [ ]:
p_body_observer = rbm.transform_points(p_body_world, outof="world", into="observer")
p_body_observer

Orientations expressed in quaternions are transformed by quaternion multiplication:
$$o_O = r \cdot o_W $$
This multiplication is implemented in the [qmul()](_generated/rigid_body_motion.qmul.rst) function to which you can pass an arbtrary number of quaternions to multiply. Assuming the body is oriented in the same direction as the world frame, transforming the orientation into the observer frame results in a rotation around the yaw axis:

In [ ]:
o_body_world = np.array((1, 0, 0, 0))
rbm.qmul(r, o_body_world)

Again, for convenience, the [transform_quaternions()](_generated/rigid_body_motion.transform_quaternions.rst) function can be used in the same way as [transform_points()](_generated/rigid_body_motion.transform_points.rst):

In [ ]:
rbm.transform_quaternions(o_body_world, outof="world", into="observer")

## Moving frames

Now, let's assume that the body moves from the origin of the world frame to the origin of the observer frame in 5 steps:

In [ ]:
p_body_world = np.zeros((5, 3))
p_body_world[:, 0] = np.linspace(0, 5, 5)
p_body_world

We can add the body positions to the plot with [plot.points()](_generated/rigid_body_motion.plot.points.rst):

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

rbm.plot.reference_frame(rf_world, ax=ax)
rbm.plot.reference_frame(rf_observer, rf_world, ax=ax)
rbm.plot.points(p_body_world, ax=ax, fmt="yo-")

We will now attach a reference frame to the moving body to explain the handling of moving reference frames. For this, we need to associate the positions of the body with corresponding timestamps:

In [ ]:
ts_body = np.arange(5)

Let's construct the moving body frame and add it to the registry. We will use the [register_frame()](_generated/rigid_body_motion.register_frame.rst) convenience method:

In [ ]:
rbm.register_frame("body", translation=p_body_world, timestamps=ts_body, parent="world")
rbm.render_tree("world")

If we transform a static point from the world into the body frame its position will change over time, which is why [transform_points()](_generated/rigid_body_motion.transform_points.rst) will return an array of points even though we pass only a single point:

In [ ]:
rbm.transform_points((2, 0, 0), outof="world", into="body")

One of the central features of the reference frame mechanism is its ability to consolidate arrays of timestamped motion even when the timestamps don't match. To illustrate this, let's create a second body moving in the y direction in world coordinates whose timestamps are offset by 0.5 seconds compared to the first body:

In [ ]:
p_body2_world = np.zeros((5, 3))
p_body2_world[:, 1] = np.linspace(0, 2, 5)
ts_body2 = ts_body - 0.5

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

rbm.plot.reference_frame(rf_world, ax=ax)
rbm.plot.reference_frame(rf_observer, rf_world, ax=ax)
rbm.plot.points(p_body_world, ax=ax, fmt="yo-")
rbm.plot.points(p_body2_world, ax=ax, fmt="co-")

Transforming the position of the second body into the frame of the first body still works, despite the timestamp mismatch:

In [ ]:
p_body2_body, ts_body2_body = rbm.transform_points(
    p_body2_world,
    outof="world",
    into="body",
    timestamps=ts_body2,
    return_timestamps=True,
)

This is because behind the scenes, [transform_points()](_generated/rigid_body_motion.transform_points.rst) matches the timestamps of the array to transform with those of the transformation across the tree by

1. computing the range of timestamps for which the transformation is defined,
2. intersecting that range with the range of timestamps to be transformed and
3. interpolating the resulting transformation across the tree to match the timestamps of the array.

Note that we specified `return_timestamps=True` to obtain the timestamps of the transformed array as they are different from the original timestamps. Let's plot the position of both bodies wrt the world frame as well as the position of the second body wrt the first body to see how the timestamp matching works:

In [ ]:
fig, axarr = plt.subplots(3, 1, sharex=True, sharey=True)

axarr[0].plot(ts_body, p_body_world, "*-")
axarr[0].set_ylabel("Position (m)")
axarr[0].set_title("First body wrt world frame")
axarr[0].grid("on")

axarr[1].plot(ts_body2, p_body2_world, "*-")
axarr[1].set_ylabel("Position (m)")
axarr[1].set_title("Second body wrt world frame")
axarr[1].grid("on")

axarr[2].plot(ts_body2_body, p_body2_body, "*-")
axarr[2].set_xlabel("Time (s)")
axarr[2].set_ylabel("Position (m)")
axarr[2].set_title("Second body wrt first body frame")
axarr[2].grid("on")
axarr[2].legend(["x", "y", "z"], loc="upper left")

fig.tight_layout()

As you can see, the resulting timestamps are the same as those of the second body; however, the first sample has been dropped because the transformation is not defined there.

## Transforming velocities

### Angular velocity

In [ ]:
from quaternion import as_float_array, from_euler_angles

r_body2_world = as_float_array(from_euler_angles(np.linspace(0, np.pi / 2, 5), 0, 0))

rbm.register_frame(
    "body2",
    translation=p_body2_world,
    rotation=r_body2_world,
    timestamps=ts_body2,
    parent="world",
)

In [ ]:
w_body2_world, ts_body2 = rbm.lookup_angular_velocity(
    moving_frame="body2", reference="world", represent_in="body2"
)
w_body2_world

$$\omega_{B2/O} = \underbrace{\omega_{B2/W}}_{\text{Input}} + \underbrace{\omega_{W/O}}_{\text{Lookup}}$$

In [ ]:
w_body2_observer = rbm.transform_angular_velocity(
    w_body2_world,
    outof="world",
    into="observer",
    timestamps=ts_body2,
    return_timestamps=False,
)
w_body2_observer

$$\omega_{B/W} = \underbrace{\omega_{B2/W}}_{\text{Input}} + \underbrace{\omega_{B/B2}}_{\text{Lookup}}$$

In [ ]:
w_body_world = rbm.transform_angular_velocity(
    w_body2_world,
    outof="body2",
    into="body",
    what="moving_frame",
    timestamps=ts_body2,
    return_timestamps=False,
)
w_body_world

### Linear velocity

In [ ]:
v_body2_world, ts_body2 = rbm.lookup_linear_velocity(
    moving_frame="body2", reference="world", represent_in="body2"
)
v_body2_world

$$v_{B2/O} = \underbrace{v_{B2/W}}_{\text{Input}} + \underbrace{v_{W/O} + \omega_{W/O} \times t_{B2/W}}_{\text{Lookup}}$$

In [ ]:
v_body2_observer = rbm.transform_linear_velocity(
    v_body2_world,
    outof="world",
    into="observer",
    moving_frame="body2",
    timestamps=ts_body2,
    return_timestamps=False,
)
v_body2_observer

$$v_{B/W} = \underbrace{v_{B2/W}}_{\text{Input}} + \underbrace{v_{B/B2} + \omega_{B2/W} \times t_{B/B2}}_{\text{Lookup}}$$

In [ ]:
v_body_world = rbm.transform_linear_velocity(
    v_body2_world,
    outof="body2",
    into="body",
    what="moving_frame",
    reference_frame="world",
    timestamps=ts_body2,
    return_timestamps=False,
)
v_body_world